In [ ]:
def show_state(env, step=0, name="", info="", image=None):
    """Fn to visualize the agent playing the game in a notebook
    """
    plt.figure(10)
    plt.clf()
    if image is not None:
        im = image
    else:
        im = env.render(mode="rgb_array")[0]
    plt.imshow(im)
    plt.title("{} | Step: {} {}".format(name, step, info))
    plt.axis('off')
    display.clear_output(wait=True)
    display.display(plt.gcf())

In [ ]:
import torch
from scipy import optimize
import torch.nn.functional as F
import numpy as np
from tqdm import tqdm
from collections import OrderedDict
import matplotlib.pyplot as plt
from IPython import display
from optimization.Optimizer import PyTorchObjective

from baselines.common.vec_env.shmem_vec_env import ShmemVecEnv
from utils.utils import zelda_spaces

from copy import deepcopy

import time
from utils.diff_evo import differential_evolution

from es import SimpleGA, OpenES

import gym

import gvgai

from generator.levels.base import Generator

import numpy as np

import ctypes as c

from agent.base import Agent


from scipy.optimize import Bounds

In [ ]:
from generator.env_gen_wrapper import GridGame
from agent.NNagent import NNagent

In [ ]:
_x = NNagent(GridGame(game='dzelda', 
                      play_length=1000, 
                      path='./levels',
                      lvl_name='5.txt',
                      mechanics=['+', 'g'], # monsters, key, door, wall
                      images=True,
                  ),
             parent=torch.load('./results/RL/map5b/best/2.0/baseline_223_best_model_policy.pt')
            )

In [ ]:
_x.nn = torch.load('./results/RL/map5b/best/2.0/baseline_59_best_model_policy.pt')

In [ ]:
_x.fitness(fn=show_state, rl=True)

In [ ]:
_x.env.env.close()

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions.categorical import Categorical
from torch.utils.tensorboard import SummaryWriter

#from cleanrl.common import preprocess_obs_space, preprocess_ac_space

import argparse
import numpy as np
import gym
import gvgai
from gym.wrappers import TimeLimit, Monitor
from gym.spaces import Discrete, Box, MultiBinary, MultiDiscrete, Space
import time
import random
import os


from generator.env_gen_wrapper import GridGame
from agent.NNagent import NNagent

In [2]:
# if __name__ == "__main__":
#     parser = argparse.ArgumentParser(description='PPO agent')
#     # Common arguments
#     parser.add_argument('--exp-name', type=str, default=os.path.basename(__file__).rstrip(".py"),
#                        help='the name of this experiment')
#     parser.add_argument('--gym-id', type=str, default="CartPole-v0",
#                        help='the id of the gym environment')
#     parser.add_argument('--learning-rate', type=float, default=7e-4,
#                        help='the learning rate of the optimizer')
#     parser.add_argument('--seed', type=int, default=1,
#                        help='seed of the experiment')
#     parser.add_argument('--episode-length', type=int, default=0,
#                        help='the maximum length of each episode')
#     parser.add_argument('--total-timesteps', type=int, default=100000,
#                        help='total timesteps of the experiments')
#     parser.add_argument('--torch-deterministic', type=bool, default=True,
#                        help='whether to set `torch.backends.cudnn.deterministic=True`')
#     parser.add_argument('--cuda', type=bool, default=True,
#                        help='whether to use CUDA whenever possible')
#     parser.add_argument('--prod-mode', type=bool, default=False,
#                        help='run the script in production mode and use wandb to log outputs')
#     parser.add_argument('--capture-video', type=bool, default=False,
#                        help='weather to capture videos of the agent performances (check out `videos` folder)')
#     parser.add_argument('--wandb-project-name', type=str, default="cleanRL",
#                        help="the wandb's project name")
#     parser.add_argument('--wandb-entity', type=str, default=None,
#                        help="the entity (team) of wandb's project")

#     # Algorithm specific arguments
#     parser.add_argument('--gamma', type=float, default=0.99,
#                        help='the discount factor gamma')
#     parser.add_argument('--vf-coef', type=float, default=0.25,
#                        help="value function's coefficient the loss function")
#     parser.add_argument('--max-grad-norm', type=float, default=0.5,
#                        help='the maximum norm for the gradient clipping')
#     parser.add_argument('--ent-coef', type=float, default=0.01,
#                        help="policy entropy's coefficient the loss function")
#     parser.add_argument('--clip-coef', type=float, default=0.2,
#                        help="the surrogate clipping coefficient")
#     parser.add_argument('--update-epochs', type=int, default=3,
#                         help="the K epochs to update the policy")
#     args = parser.parse_args()
#     if not args.seed:
#         args.seed = int(time.time())

In [3]:
expname = 'lvl4'
gym_id = 'dzelda'
lr = 7e-4
seed = 42
episode_len = 250
total_timesteps = 5000000
torch.backends.cudnn.deterministic=True

gamma = 0.99
value_coeff = 0.25
grad_norm = 0.5
ent_coeff = 0.01
clip = 0.2
epoch_update = 3


In [4]:
env = GridGame(game='dzelda', 
                      play_length=250, 
                      path='./levels',
                      lvl_name='5.txt',
                      mechanics=['+', 'g'], # monsters, key, door, wall
                      images=False,
                  )

In [5]:
# References:
# https://github.com/hill-a/stable-baselines/blob/65ed3969e8859092e32e0cf89ac42959a7f283d6/stable_baselines/common/input.py#L6
# https://github.com/hill-a/stable-baselines/blob/65ed3969e8859092e32e0cf89ac42959a7f283d6/stable_baselines/common/distributions.py#L182

import torch
import torch.nn.functional as F
from torch.distributions.categorical import Categorical
import numpy as np
from gym.spaces import Discrete, Box, MultiBinary, MultiDiscrete, Space

def preprocess_obs_space(obs_space: Space, device: str):
    """
    The `preprocess_obs_fn` receives the observation `x` in the shape of
    `(batch_num,) + obs_space.shape`.
    1) If the `obs_space` is `Discrete`, `preprocess_obs_fn` outputs a
    preprocessed obs in the shape of
    `(batch_num, obs_space.n)`.
    2) If the `obs_space` is `Box`, `preprocess_obs_fn` outputs a
    preprocessed obs in the shape of
    `(batch_num,) + obs_space.shape`.
    In addition, the preprocessed obs will be sent to `device` (either
    `cpu` or `cuda`)
    """
    if isinstance(obs_space, Discrete):
        def preprocess_obs_fn(x):
            return F.one_hot(torch.LongTensor(x), obs_space.n).float().to(device)
        return (obs_space.n, preprocess_obs_fn)

    elif isinstance(obs_space, Box):
        def preprocess_obs_fn(x):
            return torch.Tensor(x).float().view(torch.Tensor(x).shape[0], -1).to(device)
        return (np.array(obs_space.shape).prod(), preprocess_obs_fn)

    else:
        raise NotImplementedError("Error: the model does not support input space of type {}".format(
            type(obs_space).__name__))

def preprocess_ac_space(ac_space: Space):
    if isinstance(ac_space, Discrete):
        return ac_space.n

    elif isinstance(ac_space, MultiDiscrete):
        return ac_space.nvec.sum()

    elif isinstance(ac_space, Box):
        return np.prod(ac_space.shape)

    else:
        raise NotImplementedError("Error: the model does not support output space of type {}".format(
            type(ac_space).__name__))

In [6]:
from utils.utils import zelda_spaces

In [7]:
# TRY NOT TO MODIFY: setup the environment
experiment_name = f"{gym_id}__{experiment_name}__{int(time.time())}"
writer = SummaryWriter(f"./runs/{experiment_name}")
# writer.add_text('hyperparameters', "|param|value|\n|-|-|\n%s" % (
#         '\n'.join([f"|{key}|{value}|" for key, value in vars(args).items()])))
# if args.prod_mode:
#     import wandb
#     wandb.init(project=args.wandb_project_name, entity=args.wandb_entity, tensorboard=True, config=vars(args), name=experiment_name, monitor_gym=True)
#     writer = SummaryWriter(f"/tmp/{experiment_name}")
#     wandb.save(os.path.abspath(__file__))

# TRY NOT TO MODIFY: seeding
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#env = gym.make(args.gym_id)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
# torch.backends.cudnn.deterministic = args.torch_deterministic
# input_shape = env.observation_space.shape[0]
input_shape, preprocess_obs_fn = preprocess_obs_space(zelda_spaces[0], device)
output_shape = preprocess_ac_space(zelda_spaces[1])
# respect the default timelimit
# if int(episode_len):
#     if not isinstance(env, TimeLimit):
#         env = TimeLimit(env, int(args.episode_length))
#     else:
#         env._max_episode_steps = int(episode_len)
# else:
#     args.episode_length = env._max_episode_steps if isinstance(env, TimeLimit) else 200
    
# if args.capture_video:
#     env = Monitor(env, f'videos/{experiment_name}')

In [8]:
class Value(nn.Module):
    def __init__(self, depth):
        super(Value, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=depth, out_channels=8, kernel_size=3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=32, kernel_size=3)
        self.fc = nn.Linear((32 * 3) + 4, 1)

    def forward(self, x, compass):
        x = self.pool(F.relu(self.conv1(x)))
        x = F.relu(self.conv2(x))
        x = x.view(len(x), 32 * 3)
        x = torch.cat([x, compass], dim=1)
        x = F.relu(self.fc(x))
        return x

In [10]:
pg = NNagent(GG=None, 
             parent=torch.load('./results/RL/ppo_test.pt'),
             actions=6, depth=13)
vf = Value(13)
vf.double()
optimizer = optim.Adam(list(pg.nn.parameters()) + list(vf.parameters()), lr=lr)
loss_fn = nn.MSELoss()

In [ ]:
# TRY NOT TO MODIFY: start the game
global_step = 0
while global_step < total_timesteps:
    next_obs = np.array(env.reset())
    actions = np.empty((episode_len,), dtype=object)
    rewards, dones = np.zeros((2, episode_len))
    obs = np.empty((episode_len,) + zelda_spaces[0].shape)
    compass_info = np.empty((episode_len, 4))

    # ALGO LOGIC: put other storage logic here
    values = torch.zeros((episode_len), device=device)
    neglogprobs = torch.zeros((episode_len,), device=device)
    entropys = torch.zeros((episode_len,), device=device)

    # TRY NOT TO MODIFY: prepare the execution of the game.
    for step in range(episode_len):
        global_step += 1
        obs[step] = next_obs.copy()
        
        compass_info[step] = pg.compass_info

        # ALGO LOGIC: put action logic here
        values[step] = vf.forward(torch.DoubleTensor(obs[step:step+1]), 
                                  torch.DoubleTensor(compass_info[step:step+1]))

        # ALGO LOGIC: `env.action_space` specific logic
        action, neglogprob, entropy = pg.rl_get_action(torch.DoubleTensor(obs[step:step+1]), 
                                                       torch.DoubleTensor(compass_info[step:step+1]))
        
        actions[step], neglogprobs[step], entropys[step] = action.item(), neglogprob, entropy

        # TRY NOT TO MODIFY: execute the game and log data.
        next_obs, rewards[step], dones[step], _ = env.step(actions[step])
        next_obs = np.array(next_obs)
        if dones[step]:
            break

    # ALGO LOGIC: training.
    # calculate the discounted rewards, or namely, returns
    returns = np.zeros_like(rewards)
    for t in reversed(range(rewards.shape[0]-1)):
        returns[t] = rewards[t] + gamma * returns[t+1] * (1-dones[t])
    # advantages are returns - baseline, value estimates in our case
    advantages = returns - values.detach().cpu().numpy()

    neglogprobs = neglogprobs.detach()
    non_empty_idx = np.argmax(dones) + 1
    for _ in range(epoch_update):
        # ALGO LOGIC: `env.action_space` specific logic
        _, new_neglogprobs, _ = pg.rl_get_action(torch.DoubleTensor(obs[:non_empty_idx]), 
                                                 torch.DoubleTensor(compass_info[:non_empty_idx]))
        
        ratio = torch.exp(neglogprobs[:non_empty_idx] - new_neglogprobs)
        surrogate1 = ratio * torch.Tensor(advantages)[:non_empty_idx]
        surrogate2 = torch.clamp(ratio, 1-clip, 1+clip) * torch.Tensor(advantages)[:non_empty_idx]
        clip_value = torch.min(surrogate1, surrogate2)
        vf_loss = loss_fn(torch.Tensor(returns), values) * value_coeff
        loss = vf_loss - (clip_value + entropys[:non_empty_idx] * ent_coeff).mean()

        optimizer.zero_grad()
        loss.backward(retain_graph=True)
        nn.utils.clip_grad_norm_(list(pg.nn.parameters()) + list(vf.parameters()), grad_norm)
        optimizer.step()

    # TRY NOT TO MODIFY: record rewards for plotting purposes
    writer.add_scalar("charts/episode_reward", rewards.sum(), global_step)
    writer.add_scalar("losses/value_loss", vf_loss.item(), global_step)
    writer.add_scalar("losses/entropy", entropys[:step].mean().item(), global_step)

In [ ]:
env.close()
writer.close()

In [ ]:
torch.save(pg.nn, "./results/RL/ppo_test.pt")